In [ ]:
# https://www.blog.pythonlibrary.org/2018/05/03/exporting-data-from-pdfs-with-python/
# https://regex101.com/

%reset -f

import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Check directory and import csv as np.array

cwd = os.getcwd()
print(cwd,'\n\n')

import io
import re
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

import os, fnmatch
import xlrd

###

import os
# owd = 'insert_project_file_path'
# nwd = 'insert_documents_file_path'
# os.chdir(nwd)
# cwd = os.getcwd()
# os.chdir(nwd)
print('Current: ',cwd,'\nOriginal: ',owd,'\n\n')#,'\n',nwd)

###

def getInfos(currentDir):
    infos = {}
    i = 0
    for root, dirs, files in os.walk(currentDir): # Walk directory tree
        #print(root,'\n',dirs,'\n',files,'\n')
        for f in files:
            infos[i] = [f,root]
            #print(f,'\n',root,'\n\n\n')
            i+= 1
    return infos

###

# infoAI = getInfos(nwd+'/insert_first_document_set')
# infoELT = getInfos(nwd+'/insert_second_document_set')

###

def extract_text_from_pdf(pdf_path):
    #text extract from pdf
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    #close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

###

def extractCOAdocs(infos):
    count = []
    coa = {}
    j = 0
    for i in range(0,len(infos)):
        f = infos[i][0]
        if 'COA' in f:
            #print(f)
            count.append(f)
            txt = extract_text_from_pdf(infos[i][1]+'/'+infos[i][0])
            #print(txt)
            temp = [infos[i][0],infos[i][1],txt]
            coa[j] = temp
            j += 1
            #coa.append(infoAI[i].append(extract_text_from_pdf(infos[i][1]+'/'+infos[i][0])))
            #infoAI[i].append(extract_text_from_pdf(infos[i][1]+'/'+infos[i][0]))
        elif 'CoA' in f:
            #print(f)
            count.append(f)
            txt = extract_text_from_pdf(infos[i][1]+'/'+infos[i][0])
            #print(txt)
            temp = [infos[i][0],infos[i][1],txt]
            coa[j] = temp
            j += 1
        elif 'coa' in f:
            #print(f)
            count.append(f)
            txt = extract_text_from_pdf(infos[i][1]+'/'+infos[i][0])
            #print(txt)
            temp = [infos[i][0],infos[i][1],txt]
            coa[j] = temp
            j += 1
    return coa

###

def extractBatchReleaseDocs(infos):
    count = []
    coa = {}
    j = 0
    for i in range(0,len(infos)):
        f = infos[i][0]
        print(f)
        if 'pdf' in f:
            if 'Batch' and 'Release' in f:
                print('yes')
                #print(f)
                count.append(f)
                txt = extract_text_from_pdf(infos[i][1]+'/'+infos[i][0])
                #print(txt)
                temp = [infos[i][0],infos[i][1],txt]
                coa[j] = temp
                j += 1
                print(txt)
            else:
                pass
        else:
            pass

    return coa

###

def extractInfo_AI(txtBlock):
    # part number (pn)
    # flavor name (flavor)
    # nicotine concentration (nicCon) in w/w%
    # pH (ph)
    # refractive index (ri)
    # specific gravity (sg)
    # density (dens) in lb/gal
    
    txt = txtBlock
    
    try:
        lot = re.search('%(.+?)AnalysisResultProcedure', txt).group(1)
    except AttributeError:
        # AAA, ZZZ not found in the original string
        lot = '' # apply your error handling
    
    try:
        pn = re.search('INC.PL(.+?)#', txt).group(1)
        pn = pn.split('-')[0]
    except AttributeError:
        # AAA, ZZZ not found in the original string
        pn = '' # apply your error handling
    
    try:
        flavor = re.search('#(.+?)%', txt).group(1)
    except AttributeError:
        # AAA, ZZZ not found in the original string
        flavor = '' # apply your error handling
    
    try:
        nicCon = re.search('INSPECTIONNICOTINE CONTENT(.+?)PH', txt).group(1)
#         if AttributeError:
#             nicCon = re.search('INSPECTIONMOBILE(.+?)NICOTINE', txt).group(1)
#     except AttributeError:
#         nicCon = re.search('INSPECTIONMOBILE(.+?)NICOTINE', txt).group(1)
    except AttributeError:
        # AAA, ZZZ not found in the original string
        nicCon = '' # apply your error handling
    
    try:
        ph = re.search('PH(.+?)ANALYTICAL PH', txt).group(1)
    except AttributeError:
        # AAA, ZZZ not found in the original string
        ph = '' # apply your error handling
        
    try:
        ri = re.search('TESTREFRACTIVE INDEX(.+?)USP<831>', txt).group(1)
    except AttributeError:
        # AAA, ZZZ not found in the original string
        ri = '' # apply your error handling
        
    try:
        sg = re.search('SPECIFIC GRAVITY(.+?)USP<841>', txt).group(1)
    except AttributeError:
        # AAA, ZZZ not found in the original string
        sg = '' # apply your error handling
        
    try:
        dens = re.search('°C (.+?)LB/GAL', txt).group(1)
    except AttributeError:
        # AAA, ZZZ not found in the original string
        dens = '' # apply your error handling
    
    info = [lot, pn, flavor, nicCon, ph, ri, sg, dens]
#     print('\n lot number (lot) \n',
#           'part number (pn) \n',
#           'flavor name (flavor) \n',
#           'nicotine concentration (nicCon) in w/w% \n',
#           'pH (ph) \n',
#           'refractive index (ri) \n',
#           'specific gravity (sg) \n',
#           'density (dens) in lb/gal) \n')
    return info

###

# def extractInfo_ELT(txtBlock):
#     # part number (pn)
#     # flavor name (flavor)
#     # nicotine concentration (nicCon) in w/w%
#     # pH (ph)
#     # refractive index (ri)
#     # specific gravity (sg)
#     # density (dens) in lb/gal
    
#     txt = txtBlock
    
#     try:
#         lot = re.search('Lot Number(.+?) ', txt).group(1)
#     except AttributeError:
#         # AAA, ZZZ not found in the original string
#         lot = '' # apply your error handling
    
#     try:
#         pn = '86'+re.search('86(.+?) ', txt).group(1)
#         print(pn)
#     except AttributeError:
#         # AAA, ZZZ not found in the original string
#         pn = '' # apply your error handling
    
#     try:
#         flavor = re.search('ANALYSIS(.+?)86', txt).group(1)
#     except AttributeError:
#         # AAA, ZZZ not found in the original string
#         flavor = '' # apply your error handling
    
#     try:
#         nicCon = re.search('NICOTINE(.+?)pH', txt).group(1)
#     except AttributeError:
#         # AAA, ZZZ not found in the original string
#         nicCon = '' # apply your error handling
    
#     try:
#         ph = re.search('pH(.+?) \d', txt).group(1)
#     except AttributeError:
#         # AAA, ZZZ not found in the original string
#         ph = '' # apply your error handling
        
#     try:
#         ri = re.search('REFRACTIVE INDEX(.+?)NICOTINE', txt).group(1)
#     except AttributeError:
#         # AAA, ZZZ not found in the original string
#         ri = '' # apply your error handling
        
#     try:
#         sg = re.search('SPECIFIC GRAVITY(.+?)REFRACTIVE INDEX', txt).group(1)
#     except AttributeError:
#         # AAA, ZZZ not found in the original string
#         sg = '' # apply your error handling
    
#     info = [lot, pn, flavor, nicCon, ph, ri, sg]
#     print('\n lot number (lot) \n',
#           'part number (pn) \n',
#           'flavor name (flavor) \n',
#           'nicotine concentration (nicCon) in w/w% \n',
#           'pH (ph) \n',
#           'refractive index (ri) \n',
#           'specific gravity (sg) \n')
#     return info

def extractInfo_ELT(txtBlock):
    
    txt = txtBlock
    
    # lot number
    try:
        lot = re.search('Lot Number(.+?) ', txt).group(1)
        lot = lot.replace('Customer','')
        lot = lot.replace(':','')
        lot = lot.replace(' ','')
    except AttributeError:
        lot = '' 
    # part number
    try:
        pn = '86EL'+re.search('86EL(.+?) ', txt).group(1)
        pn = pn.replace('Customer','')
    except AttributeError:
        pn = '' 
    # flavor name 
    try:
        flavor = re.search('ANALYSIS(.+?)86', txt).group(1)
        if 'Date' in flavor:
            flavor = re.search('Customer:(.+?)86', txt).group(1)
        flavor = flavor.replace('#','')
        flavor = flavor.strip()
    except AttributeError:
        flavor = '' 
    # nicotine concentration
    try:
        temp = re.search('NICOTINE(.+?)pH', txt).group(1)
        if 'w:' in temp:
            temp = temp.split(':')[1]
            nicCon = temp.replace(' ','')
        else:
            temp = re.search('%(.+?)\d.\d\d to', temp).group(1)
            nicCon = temp.replace(' ','')
        nicCon = nicCon.replace('%','')
    except AttributeError:
        nicCon = '' 
    # pH
    try:
        temp = re.search('10%(.+?)C', txt).group(1)
        if 'SHELF' in temp:
            
            temp = re.search(': (.+?) S', temp).group(1)
            ph = temp.replace(' ', '')
        else:
            
            temp = re.search('WATER(.+?)to',temp).group(1)
            ph = temp[0:3]
    except AttributeError:
        ph = '' 
    # refractive index
    try:
        temp = re.search('REFRACTIVE INDEX(.+?)NICOTINE', txt).group(1)
        #print('Refractive index string: ',temp)
        if 'to' in temp:
            temp = re.search('C(.+?)to',temp).group(1)
            ri = temp[0:5]
        else:
            temp = temp.split(':')[1]
            ri = temp.replace(' ','')
    except AttributeError:
        ri = '' 
    # specific gravity
    try:
        temp = re.search('SPECIFIC GRAVITY(.+?)REFRACTIVE INDEX', txt).group(1)
        #print('Specific gravity string: ',temp)
        if 'to' in temp:
            temp = re.search('C(.+?)to',temp).group(1)
            sg = temp[0:5]
        else:
            temp = temp.split(':')[1]
            sg = temp.replace(' ','')
    except AttributeError:
        sg = '' 
    # Ethyl alcohol
    try:
        eth = re.search('Ethyl (.+?)%Room', txt).group(1)
        eth = eth[-4:]
    except AttributeError:
        eth = '' 
    # Flash Point
    try:
        flash = re.search('Flash Point:(.+?)Ethyl', txt).group(1)
    except AttributeError:
        flash = '' 

    info = [lot, pn, flavor, nicCon, ph, ri, sg, eth, flash]
    return info


###

aiCOA = extractCOAdocs(infoAI)
eltCOA = extractCOAdocs(infoELT)

for i in range(0,len(aiCOA)):
    txt = aiCOA[i][2]
    newInfo = extractInfo_AI(txt)
    aiCOA[i].append(newInfo)
    #print(aiCOA[i])
    
for i in range(0,len(eltCOA)):
    txt = eltCOA[i][2]
    newInfo = extractInfo_ELT(txt)
    #print(newInfo)
    eltCOA[i].append(newInfo)
    
# aiBR = extractBatchReleaseDocs(infoAI)
# eltBR = extractBatchReleaseDocs(infoELT)